# <font size="6">Libraries</font>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# <font size="5">dataset</font>

In [23]:
train_df = pd.read_parquet('data/train.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

valid_df = pd.read_parquet('data/valid.parquet').drop(columns=['transaction_number',
                                                               'reversed_transaction', 
                                                               'transaction_max',
                                                               'time'])

In [24]:
len(np.unique(train_df['app_id'])), len(np.unique(valid_df['app_id']))

(430309, 429711)

In [25]:
train_df

,app_id,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins
0,0,0.465425,2,-1,3,8
1,0,0.000000,2,0,0,5
2,0,0.521152,2,0,3,8
3,0,0.356078,10,52,3,40
4,0,0.000000,2,280,0,5
...,...,...,...,...,...,...
95046144,1003049,0.227264,108,0,0,429
95046145,1003049,0.387565,57,0,3,228
95046146,1003049,0.273395,1,1,0,1
95046147,1003049,0.258972,2,19,0,5


In [26]:
train_df['mcc_next'] = train_df.groupby(['app_id'])['mcc'].shift(-1)
train_df['amnt_next'] = train_df.groupby(['app_id'])['amnt'].shift(-1)
train_df['hour_diff_next'] = train_df.groupby(['app_id'])['hour_diff'].shift(-1)
train_df['amnt_bins_next'] = train_df.groupby(['app_id'])['amnt_bins'].shift(-1)
train_df['amnt_mcc_bins_next'] = train_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

valid_df['mcc_next'] = valid_df.groupby(['app_id'])['mcc'].shift(-1)
valid_df['amnt_next'] = valid_df.groupby(['app_id'])['amnt'].shift(-1)
valid_df['hour_diff_next'] = valid_df.groupby(['app_id'])['hour_diff'].shift(-1)
valid_df['amnt_bins_next'] = valid_df.groupby(['app_id'])['amnt_bins'].shift(-1)
valid_df['amnt_mcc_bins_next'] = valid_df.groupby(['app_id'])['amnt_mcc_bins'].shift(-1)

In [27]:
last_transactions_train = train_df.groupby('app_id').tail(1)

first_transactions_valid = valid_df.groupby('app_id').head(1)
first_transactions_valid = first_transactions_valid.dropna()

# <font size="6">Catboost [MCC, amount] -> hour_diff</font>

In [27]:
train_df = train_df.dropna()
valid_df = valid_df.dropna()

In [29]:
import catboost
from catboost import CatBoostRegressor

regressor = CatBoostRegressor(
    learning_rate=1e-1,
    iterations=2000,
    depth=2,
    verbose=100,
    task_type='GPU',
    loss_function='MAE',
    eval_metric='MAE',
    early_stopping_rounds=100
)

regressor.fit(train_df[['amnt', 'mcc', 'hour_diff', 'amnt_bins', 'amnt_mcc_bins']],
              train_df['hour_diff_next'])

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 25.9005792	total: 637ms	remaining: 21m 14s
100:	learn: 25.7599056	total: 1m 3s	remaining: 19m 52s
200:	learn: 25.6458369	total: 2m 6s	remaining: 18m 53s
300:	learn: 25.5566117	total: 3m 9s	remaining: 17m 49s
400:	learn: 25.4897382	total: 4m 11s	remaining: 16m 42s
500:	learn: 25.4397292	total: 5m 12s	remaining: 15m 34s
600:	learn: 25.4022826	total: 6m 17s	remaining: 14m 38s
700:	learn: 25.3733886	total: 7m 22s	remaining: 13m 39s
800:	learn: 25.3509315	total: 8m 30s	remaining: 12m 44s
900:	learn: 25.3327710	total: 9m 38s	remaining: 11m 46s
1000:	learn: 25.3187637	total: 10m 50s	remaining: 10m 49s
1100:	learn: 25.3062824	total: 12m 1s	remaining: 9m 48s
1200:	learn: 25.2954841	total: 13m 13s	remaining: 8m 48s
1300:	learn: 25.2865472	total: 14m 31s	remaining: 7m 47s
1400:	learn: 25.2782408	total: 15m 52s	remaining: 6m 47s
1500:	learn: 25.2709219	total: 17m 16s	remaining: 5m 44s
1600:	learn: 25.2647340	total: 18m 43s	remaining: 4m 39s
1700:	learn: 25.2592307	total: 20m 10s	remainin

In [34]:
predicted_times = regressor.predict(last_transactions_train[['amnt',
                                                       'mcc', 
                                                       'hour_diff',
                                                       'amnt_bins',
                                                       'amnt_mcc_bins']].values.reshape(-1, 5))

predicted_times = pd.DataFrame({'app_id': last_transactions_train['app_id'].values, 'time_pred': predicted_times})

In [35]:
dataframe = predicted_times.merge(first_transactions_valid, on='app_id', how='inner')

In [36]:
from sklearn.metrics import mean_absolute_error, accuracy_score

print('Time MAE:', mean_absolute_error(dataframe['hour_diff_next'], dataframe['time_pred']))

Time MAE: 30.785762689699595


# <font size="6">Linear regression [MCC, amnt] -> hour_diff </font>

In [7]:
first_transactions_valid['hour_diff'].mean()

32.944784001344736

In [169]:
import sklearn
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(train_df[['amnt', 'mcc', 'hour_diff']],
              train_df['hour_diff_next'])

predicted_times = regressor.predict(last_transactions_train[['amnt', 'mcc', 'hour_diff']].values.reshape(-1, 3))
predicted_times = pd.DataFrame({'app_id': last_transactions_train['app_id'].values, 'time_pred': predicted_times})

/home/user/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [172]:
dataframe = predicted_times.merge(first_transactions_valid, on='app_id', how='inner')

In [173]:
from sklearn.metrics import mean_absolute_error, accuracy_score

print('Time MAE:', mean_absolute_error(dataframe['hour_diff_next'], dataframe['time_pred']))

Time MAE: 34.44031655887703


# <font size="6">Mean time по всему train датасету</font>

In [ ]:
train_df['hour_diff'].median()

In [ ]:
first_transactions_valid['time_pred'] = train_df['hour_diff'].median()

from sklearn.metrics import mean_absolute_error, accuracy_score

print('Time MAE:', mean_absolute_error(first_transactions_valid['hour_diff'], first_transactions_valid['time_pred']))

# <font size="6">Mean time по app_id train датасету</font>

In [18]:
avg_time = train_df.groupby('app_id')['hour_diff'].median()

avg_time = pd.DataFrame({'app_id': avg_time.index, 'hour_diff_pred': avg_time.values})

In [19]:
dataframe = avg_time.merge(first_transactions_valid, on='app_id', how='inner')

In [20]:
dataframe

,app_id,hour_diff_pred,amnt,mcc,hour_diff,amnt_bins,amnt_mcc_bins,hour_summed,transaction_min,mcc_next,amnt_next,hour_diff_next,amnt_bins_next,amnt_mcc_bins_next,time_pred
0,0,20.0,0.336345,2,0,2,7,0,132,20.0,0.359050,74.0,3.0,80.0,27.355142
1,1,12.0,0.439060,2,37,3,8,37,307,1.0,0.323313,0.0,1.0,2.0,27.355142
2,2,13.0,0.320805,28,156,1,110,156,180,2.0,0.485295,10.0,3.0,8.0,27.355142
3,4,9.0,0.387677,9,73,3,36,73,68,9.0,0.393020,0.0,3.0,36.0,27.355142
4,6,1.0,0.411448,58,24,3,232,24,211,8.0,0.384802,1.0,3.0,32.0,27.355142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428331,1003041,6.0,0.281539,3,17,0,9,17,274,1.0,0.216285,109.0,0.0,1.0,27.355142
428332,1003044,6.5,0.334992,4,0,2,15,0,211,3.0,0.333474,21.0,2.0,11.0,27.355142
428333,1003047,5.0,0.516270,15,42,3,60,42,211,15.0,0.498168,0.0,3.0,60.0,27.355142
428334,1003048,7.0,0.589482,12,143,3,48,143,170,2.0,0.583667,0.0,3.0,8.0,27.355142


In [21]:
from sklearn.metrics import mean_absolute_error, accuracy_score

print('Time MAE:', mean_absolute_error(dataframe['hour_diff_pred'], dataframe['hour_diff_next']))

Time MAE: 30.397406475290428
